In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import glob
from sklearn.feature_selection import mutual_info_classif as MIC
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
import time

In [2]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.ensemble import RandomForestClassifier

In [3]:
# number = int(get_ipython().user_ns['param'])
number = 4

In [4]:
model_acc=[]
model_precision=[]
model_recal=[]
model_f1=[]
model_nam=[]
model_confusion=[]
model_time=[]
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report,f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
df = pd.read_csv('../Dataset CSV/csv_result-data'+str(number)+' Sampled Scenarios.csv')

In [6]:
# Perform the operation to create the new feature
df['R1-VCM1'] = (df['R1-PM1:V'] - df['R1-PM7:V']) / (df['R1-PM4:I'] - df['R1-PM10:I'])
df['R3-sV1'] = np.sin(df['R3-PA1:VH'] - df['R3-PA7:VH'])
df['R3-sV3'] = np.sin(df['R3-PA3:VH'] - df['R3-PA9:VH'])
# df['R2-VCM2'] = (df['R2-PM2:V'] - df['R2-PM8:V']) / (df['R2-PM5:I'] - df['R2-PM11:I'])
df['R2-VCM1'] = (df['R2-PM1:V'] - df['R2-PM7:V']) / (df['R2-PM4:I'] - df['R2-PM10:I'])
df['R3-VCM1'] = (df['R3-PM1:V'] - df['R3-PM7:V']) / (df['R3-PM4:I'] - df['R3-PM10:I'])
df['R4-sI1'] = np.sin(df['R4-PA4:IH'] - df['R4-PA10:IH'])
df['R2-VCA4'] = df['R2-PA7:VH'] - df['R2-PA10:IH']
df['R1-sI1'] = np.sin(df['R1-PA4:IH'] - df['R1-PA10:IH'])
df['R4-VCM2'] = (df['R4-PM2:V'] - df['R4-PM8:V']) / (df['R4-PM5:I'] - df['R4-PM11:I'])
df['R2-VCA1'] = np.sin(df['R2-PA1:VH'] - df['R2-PA4:IH'] - df['R2-PA7:VH'] - df['R2-PA10:IH'])
df['R3-VCM2'] = (df['R3-PM2:V'] - df['R3-PM8:V']) / (df['R3-PM5:I'] - df['R3-PM11:I'])
df['R1-VCA4'] = df['R1-PA7:VH'] - df['R1-PA10:IH']
df['R1-VCM2'] = (df['R1-PM2:V'] - df['R1-PM8:V']) / (df['R1-PM5:I'] - df['R1-PM11:I'])
df['R2-VCM2'] = (df['R2-PM2:V'] - df['R2-PM8:V']) / (df['R2-PM5:I'] - df['R2-PM11:I'])
df['R4-VCA4'] = df['R4-PA7:VH'] - df['R4-PA10:IH']
df['R4-VCM1'] = (df['R4-PM1:V'] - df['R4-PM7:V']) / (df['R4-PM4:I'] - df['R4-PM10:I'])

In [7]:
# add the marker to the last of the list
cols = list(df.columns)
cols.remove('marker')
df = df[cols + ['marker']]

In [8]:
df.replace([np.inf, -np.inf], np.nan, inplace= True)
df.isnull().sum().sum()

3209

In [9]:
df = df.fillna(value=df.mean())
# df = df.fillna(value=0)

In [10]:
print(df.isin([np.inf, -np.inf]).sum().sum(), df.isnull().sum().sum())

0 0


In [11]:
df_z_scaled = df.copy()
for column in df_z_scaled.columns:
#     print(df_z_scaled[column])
    if (df_z_scaled[column].std() != 0) and (column != 'id') and (column != 'marker'):
        df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()     
# print(df_z_scaled)

In [12]:
X = df_z_scaled.iloc[:,1:-1]
y = df_z_scaled.iloc[:,-1:]

In [14]:
# Get the names of the top 31 features
top_n_feature_names = ['R1-PM5:I', 'R1-VCM1', 'R3-PM5:I', 'R1-PA1:VH', 'R4-PA2:VH', 'R3-sV3',
 'R1-PA3:VH', 'R2-VCM1', 'R4-VCM1', 'R3-PA5:IH', 'R1-PA2:VH', 'R3-sV1',
 'R2-PA5:IH', 'R2-PA3:VH', 'R2-PA6:IH', 'R2-PA1:VH', 'R3-VCM1', 'R4-PA5:IH',
 'R2-VCA4', 'R3-PA6:IH', 'R4-PM2:V', 'R2-PA4:IH', 'R3-PA4:IH', 'R1-PA5:IH',
 'R1-sI1', 'R2-PA2:VH', 'R1-PM2:V', 'R3-PA:ZH', 'R2-VCM2', 'R1-VCM2', 'R4-VCM2']

# Filter the original dataframe to only include the top n features
X_top_n = X[top_n_feature_names].copy()

X_new = X_top_n.copy()

In [15]:
X_new

,R1-PM5:I,R1-VCM1,R3-PM5:I,R1-PA1:VH,R4-PA2:VH,R3-sV3,R1-PA3:VH,R2-VCM1,R4-VCM1,R3-PA5:IH,...,R2-PA4:IH,R3-PA4:IH,R1-PA5:IH,R1-sI1,R2-PA2:VH,R1-PM2:V,R3-PA:ZH,R2-VCM2,R1-VCM2,R4-VCM2
0,0.627657,0.032498,0.593858,0.280064,-1.237048,-1.315449,1.213908,0.028357,-0.014322,0.632234,...,1.636321,1.636498,-1.307082,1.382465,-1.318864,-0.212841,2.724037,-0.124961,-0.055162,-0.027867
1,0.777983,0.063392,0.741773,0.264425,-1.251975,0.989595,1.196701,0.025628,-0.089056,0.611412,...,1.602650,1.602326,-1.328207,0.727325,-1.336730,-0.140439,-0.446013,-0.145185,-0.065000,-0.029703
2,0.775558,0.032498,0.739406,0.263535,-1.252769,1.045699,1.195905,0.014827,-0.054766,0.610133,...,1.598915,1.600499,-1.328912,0.617168,-1.337346,-0.119145,-0.445742,-0.145577,-0.065091,-0.029708
3,0.770709,0.032498,0.731123,0.262589,-1.253669,1.123263,1.194741,0.008040,-0.153825,0.609098,...,1.591954,1.593194,-1.330482,0.417324,-1.337962,-0.072297,-0.437863,-0.145104,-0.065039,-0.029698
4,0.771921,0.032498,0.729940,0.261698,-1.254675,1.188072,1.193639,0.009062,-0.177378,0.607333,...,1.587201,1.589305,-1.331999,0.218662,-1.338732,-0.051002,-0.436334,-0.145753,-0.065393,-0.029765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,-0.679210,0.032498,-0.564608,1.445745,-0.126369,-0.462768,-1.350914,0.030555,-0.013022,1.924613,...,-0.815532,-0.854336,-0.072547,1.631611,-0.169837,0.106578,-0.430800,0.159554,0.080392,-0.005228
5198,-0.671936,0.032498,-0.572891,1.444799,-0.127534,-0.614585,-1.351955,0.019059,-0.013022,1.925709,...,-0.822040,-0.854454,-0.078722,1.623928,-0.170967,0.110836,-0.431232,0.153000,0.078757,-0.005356
5199,-0.617382,0.032498,-0.584724,1.441515,-0.131663,-1.055520,-1.355568,0.016193,-0.061298,1.924369,...,-0.849712,-0.855868,-0.106022,1.347340,-0.174612,0.153426,-0.427190,0.126257,0.067752,-0.005694
5200,-0.568890,0.032498,-0.569341,1.438009,-0.135421,-1.348203,-1.359425,-0.032542,-0.283365,1.909331,...,-0.875573,-0.870125,-0.130614,0.341028,-0.178257,0.183238,-0.409488,0.103326,0.054063,-0.007536


In [16]:
X_train,X_test,y_train,y_test=train_test_split(X_new, y, test_size=0.1, random_state=1)

In [17]:
from sklearn.ensemble import AdaBoostClassifier

# base_model = DecisionTreeClassifier(max_depth=5)

start_time = time.time()

clf_4= RandomForestClassifier(n_estimators=100, random_state=43)
# create the AdaBoost classifier
clf_ada_dt = AdaBoostClassifier(base_estimator=clf_4)

# train the model using the training data
clf_ada_dt.fit(X_train, y_train)

# make predictions on the testing data
y_pred = clf_ada_dt.predict(X_test)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

# evaluate the model's accuracy
score = accuracy_score(y_test, y_pred)

model_precision.append(precision_score(y_test,y_pred,average='weighted'))
model_recal.append(recall_score(y_test,y_pred,average='weighted'))
model_f1.append(f1_score(y_test,y_pred,average='weighted'))
print(f'Accuracy: {round(score*100,2)}%')
model_acc.append(score)
model_time.append(elapsed_time)
model_nam.append("AdaBoost")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Elapsed time: 3.28 seconds
Accuracy: 95.2%


In [18]:
data = {'Accuracy': model_acc, 'Precision': model_precision, 'Recall': model_recal, 'f1_score': model_f1, "elapsed_time": model_time}
df1 = pd.DataFrame(data)
print(df1)
# df1.to_csv('final_mean_'+str(number)+'.csv', index=False)

   Accuracy  Precision    Recall  f1_score  elapsed_time
0  0.952015   0.955778  0.952015  0.952067      3.278566


# Confusion Matrix

In [19]:
# For total dataset
df_cm_total = y['marker'].value_counts().sort_index(ascending=True)
print(df_cm_total)

1     138
2     110
3     171
4     235
5     353
6     197
7      89
8     144
9      13
10    181
11    115
12     77
13     67
14    126
15     86
16    138
17     87
18     40
19    164
20     42
21    104
22     74
23    126
24    169
25     65
26    130
27     34
28     40
29    254
30     65
35    289
36    332
37    236
38    134
39     93
40     81
41    403
Name: marker, dtype: int64


In [20]:
# For Train Set
df_cm_train = y_train['marker'].value_counts().sort_index(ascending=True)
print(df_cm_train)

1     122
2     103
3     159
4     218
5     314
6     171
7      77
8     130
9      11
10    164
11    103
12     71
13     61
14    111
15     78
16    125
17     74
18     34
19    153
20     36
21     91
22     66
23    113
24    147
25     55
26    123
27     31
28     36
29    228
30     61
35    256
36    293
37    210
38    124
39     83
40     74
41    375
Name: marker, dtype: int64


In [21]:
# For test set
df_cm_test = y_test['marker'].value_counts().sort_index(ascending=True)
print(df_cm_test)

1     16
2      7
3     12
4     17
5     39
6     26
7     12
8     14
9      2
10    17
11    12
12     6
13     6
14    15
15     8
16    13
17    13
18     6
19    11
20     6
21    13
22     8
23    13
24    22
25    10
26     7
27     3
28     4
29    26
30     4
35    33
36    39
37    26
38    10
39    10
40     7
41    28
Name: marker, dtype: int64


In [22]:
# initialize the confusion matrix with zeros
cm = np.zeros((42, 42), dtype=int)

# fill the confusion matrix with the actual counts
for i in range(len(y_test)):
    cm[y_test.iloc[i]['marker'], y_pred[i]] += 1

In [48]:
cm

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0, 15,  0, ...,  0,  0,  0],
       [ 0,  0,  7, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  9,  0,  0],
       [ 0,  0,  0, ...,  0,  6,  0],
       [ 0,  0,  0, ...,  0,  0, 28]])

In [23]:
# get the number of classes
n_classes = 42

# initialize arrays to store the TP, FP, TN, and FN values
TP = np.zeros(n_classes, dtype=int)
FP = np.zeros(n_classes, dtype=int)
TN = np.zeros(n_classes, dtype=int)
FN = np.zeros(n_classes, dtype=int)

# print('Class', 'TP', 'FP', 'FN', 'TN')
# calculate the TP, FP, TN, and FN values for each class
for i in y_test['marker'].sort_index(ascending=False).unique():
    TP[i] = cm[i, i]
    FP[i] = np.sum(cm[:, i]) - cm[i, i]
    FN[i] = np.sum(cm[i, :]) - cm[i, i]
    TN[i] = np.sum(cm) - TP[i] - FP[i] - FN[i]

data_cm = {'Class' : range(0,42), 'TP' : TP, 'FP': FP, 'FN': FN, 'TN' : TN}
df_cm = pd.DataFrame(data_cm)
#     print(i,TP[i],FP[i],FN[i],TN[i])
df_cm = df_cm.drop([0, 31,32,33,34]).reset_index(drop=True)
print(df_cm)

    Class  TP  FP  FN   TN
0       1  15   3   1  502
1       2   7   0   0  514
2       3  12   1   0  508
3       4  17   2   0  502
4       5  36   3   3  479
5       6  25   0   1  495
6       7   9   0   3  509
7       8  14   0   0  507
8       9   2   0   0  519
9      10  16   2   1  502
10     11  12   0   0  509
11     12   5   0   1  515
12     13   5   0   1  515
13     14  14   0   1  506
14     15   8   0   0  513
15     16  13   2   0  506
16     17  12   0   1  508
17     18   6   0   0  515
18     19  11   2   0  508
19     20   6   0   0  515
20     21  12   0   1  508
21     22   7   0   1  513
22     23  11   0   2  508
23     24  21   0   1  499
24     25   9   0   1  511
25     26   6   1   1  513
26     27   3   0   0  518
27     28   4   0   0  517
28     29  26   1   0  494
29     30   4   1   0  516
30     35  33   1   0  487
31     36  37   1   2  481
32     37  25   4   1  491
33     38  10   0   0  511
34     39   9   0   1  511
35     40   6   1   1  513
3

In [24]:
TCM = np.zeros((2,2), dtype=int)
TTP = TP.sum()
TFP = FP.sum()
TFN = FN.sum()
TTN = TN.sum()
TCM[0,0] = TTP
TCM[0,1] = TFP
TCM[1,0] = TFN
TCM[1,1] = TTN
print(TCM)

[[  496    25]
 [   25 18731]]


# Finding Misclassified Data

In [47]:
y_pred1 = []
y_act = []
# Find the indices of the misclassified samples
misclassified_indices = []
for i in range(len(y_test)):
    if y_test.iloc[i]['marker'] in [7,8,9,10,11,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,35,36,37,38,39,40]:
        if y_test.iloc[i]['marker'] != y_pred[i]:
            y_pred1.append(y_pred[i])
            y_act.append(y_test.iloc[i]['marker'])
            misclassified_indices.append(i)

# Get the misclassified data from the test set
misclassified_data = X_test.iloc[misclassified_indices]
mis_df = pd.DataFrame(misclassified_data.reset_index(drop=True))
mis_df['Actual Class'] = y_act
mis_df['Predicted Class'] = y_pred1
# print(misclassified_data.reset_index(drop=True))
print(mis_df)

    R1-PM5:I       R1-VCM1  R3-PM5:I  R1-PA1:VH  R4-PA2:VH    R3-sV3  \
0  -0.422201  4.022153e-02 -0.439176   0.459438  -1.066174  0.500983   
1  -0.108213  3.249814e-02 -0.230913  -0.159661  -1.651756 -1.255278   
2   0.476118 -4.272086e-02  0.442394   1.800986   0.208407  0.978399   
3   0.365798 -7.562933e-02  0.367845  -0.767073   1.096746 -1.425442   
4   0.397318 -1.136068e-01 -0.488875  -0.902647   1.157596 -0.751528   
5   0.445811  3.494169e-02  0.281463  -0.475110   1.374301  0.631271   
6   0.478543  3.249814e-02  0.454227  -1.310148   0.576605 -1.441776   
7  -0.590711  3.995520e-02 -0.603657  -0.602057   1.250095 -1.393994   
8   0.573103 -4.134493e-02  0.538242  -0.593709   1.256763  0.961966   
9  -2.595866  3.254279e-16  1.448212  -0.432924   1.182074 -1.309134   
10  1.603563 -1.342132e-01 -0.758671  -1.095211   0.885798 -1.439841   
11 -2.595866  3.254279e-16  2.396048  -0.666783   0.801378 -1.051828   
12 -2.456451  5.556531e-02  0.152482  -1.320555   0.649785  0.32

In [26]:
len(y_act)

25

In [27]:
accuracy = (cm.diagonal().sum()) / cm.sum()
accuracy

0.9520153550863724

In [28]:
# accuracy = (TTP + TTN) / (TTP + TTN + TFP + TFN )
# accuracy

In [29]:
# print(cm.diagonal().sum() , cm.sum())
# print(TP.sum()+TN.sum(),FP.sum() + FN.sum(),TP.sum() + TN.sum() + FP.sum() + FN.sum())

In [30]:
precision = TTP / (TTP + TFP)
precision

0.9520153550863724

In [31]:
recall = TTP / (TTP + TFN)
recall

0.9520153550863724

In [32]:
f1_score = 2*(precision*recall) / (precision+recall)
f1_score

0.9520153550863724